In [67]:
import pandas as pd
import numpy as np
import hvplot.pandas
from sklearn.cluster import KMeans
import plotly.express as px
import psycopg2 as pg
import sqlalchemy
import matplotlib as plt
from sqlalchemy import create_engine
import pandas.io.sql as psql
import sklearn as skl
import tensorflow as tf

In [68]:
#Read clean vaersdata dataframe from postgreSQL 

connection = pg.connect(host='team-11.ch3wmigorcf0.us-east-2.rds.amazonaws.com', dbname="postgres", user="postgres", password='UTB00tcamp')
#dataframe = psql.DataFrame("SELECT * FROM category", connection)
VAERS_data_clean_df = pd.read_sql_query('select * from public."clean_vaers_data"',con=connection)
VAERS_data_clean_df.head()

,VAERS_ID,AGE_YRS,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,GENDER_0,GENDER_1
0,916600,33,0,0,1,1,1,1,1,0
1,916601,73,0,0,1,1,1,1,1,0
2,916602,23,0,0,1,1,1,1,1,0
3,916603,58,0,0,1,1,1,1,1,0
4,916604,47,0,0,1,1,0,1,1,0


In [69]:
#Read clean vaersvax dataframe from postgreSQL 

connection = pg.connect(host='team-11.ch3wmigorcf0.us-east-2.rds.amazonaws.com', dbname="postgres", user="postgres", password='UTB00tcamp')
#dataframe = psql.DataFrame("SELECT * FROM category", connection)
VAERS_vax_clean_df = pd.read_sql_query('select * from public."vaers_vax_cleaned_supervised"',con=connection)
VAERS_vax_clean_df.head()

,VAERS_ID,VAX_MANU
0,916600,MODERNA
1,916601,MODERNA
2,916602,PFIZERBIONTECH
3,916603,MODERNA
4,916604,MODERNA


In [70]:
VAERS_vax_clean_df = VAERS_vax_clean_df.drop_duplicates(subset= ['VAERS_ID']).copy()
VAERS_vax_clean_df.head()

,VAERS_ID,VAX_MANU
0,916600,MODERNA
1,916601,MODERNA
2,916602,PFIZERBIONTECH
3,916603,MODERNA
4,916604,MODERNA


In [71]:
VAERS_data_vax_supervised_df= pd.merge(VAERS_data_clean_df, VAERS_vax_clean_df, on = "VAERS_ID")
VAERS_data_vax_supervised_df.head()

,VAERS_ID,AGE_YRS,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,GENDER_0,GENDER_1,VAX_MANU
0,916600,33,0,0,1,1,1,1,1,0,MODERNA
1,916601,73,0,0,1,1,1,1,1,0,MODERNA
2,916602,23,0,0,1,1,1,1,1,0,PFIZERBIONTECH
3,916603,58,0,0,1,1,1,1,1,0,MODERNA
4,916604,47,0,0,1,1,0,1,1,0,MODERNA


In [72]:
VAERS_data_vax_supervised_df = pd.get_dummies(VAERS_data_vax_supervised_df, prefix=["VAX_MANU"], columns= ["VAX_MANU"]).copy()
VAERS_data_vax_supervised_df.head()

,VAERS_ID,AGE_YRS,DIED,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,GENDER_0,GENDER_1,VAX_MANU_JANSSEN,VAX_MANU_MODERNA,VAX_MANU_PFIZERBIONTECH,VAX_MANU_UNKNOWN MANUFACTURER
0,916600,33,0,0,1,1,1,1,1,0,0,1,0,0
1,916601,73,0,0,1,1,1,1,1,0,0,1,0,0
2,916602,23,0,0,1,1,1,1,1,0,0,0,1,0
3,916603,58,0,0,1,1,1,1,1,0,0,1,0,0
4,916604,47,0,0,1,1,0,1,1,0,0,1,0,0


In [73]:
### Logistic Regression Model to predict death
y = VAERS_data_vax_supervised_df["DIED"]
X = VAERS_data_vax_supervised_df.drop(["DIED"], axis=1)
X.head()

,VAERS_ID,AGE_YRS,HOSPITAL,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,GENDER_0,GENDER_1,VAX_MANU_JANSSEN,VAX_MANU_MODERNA,VAX_MANU_PFIZERBIONTECH,VAX_MANU_UNKNOWN MANUFACTURER
0,916600,33,0,1,1,1,1,1,0,0,1,0,0
1,916601,73,0,1,1,1,1,1,0,0,1,0,0
2,916602,23,0,1,1,1,1,1,0,0,0,1,0
3,916603,58,0,1,1,1,1,1,0,0,1,0,0
4,916604,47,0,1,1,0,1,1,0,0,1,0,0


In [74]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=1, stratify=y)

In [75]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
   max_iter=200,
   random_state=1)

In [76]:
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [77]:
y_pred = classifier.predict(X_test)

In [78]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9548804137039432


In [60]:
## Logistic Regression Model to predict Hospitalization using DataFrame connected to PostgreSQL Database
y = VAERS_data_vax_supervised_df["HOSPITAL"].copy()
X = VAERS_data_vax_supervised_df.drop(["HOSPITAL"], axis=1).copy()
X.head()

,VAERS_ID,AGE_YRS,DIED,OTHER_MEDS,CUR_ILL,HISTORY,ALLERGIES,GENDER_0,GENDER_1,VAX_MANU_JANSSEN,VAX_MANU_MODERNA,VAX_MANU_PFIZERBIONTECH,VAX_MANU_UNKNOWN MANUFACTURER
0,916600,33,0,1,1,1,1,1,0,0,1,0,0
1,916601,73,0,1,1,1,1,1,0,0,1,0,0
2,916602,23,0,1,1,1,1,1,0,0,0,1,0
3,916603,58,0,1,1,1,1,1,0,0,1,0,0
4,916604,47,0,1,1,0,1,1,0,0,1,0,0


In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=1, stratify=y)

In [62]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
   max_iter=200,
   random_state=1)

In [63]:
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [64]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.8881706528765352
